In [1]:

import openai
import os
import pandas as pd
from bert_score import score as bert_score


/gpuhome/shanto1/miniconda/envs/codellama/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import openai

client = openai.OpenAI(api_key="")

import os
import pandas as pd

In [ ]:

#Input data (Input file from "APU Task Extention (CodeGen) Input")
# df=pd.read_csv("")
df=pd.read_csv("")

In [4]:
df.head()

,Unnamed: 0.1,Unnamed: 0,rag_pred,gt,PostId,code_before,code_after,AcceptanceDate,LastEditDate,EditorId,...,comment,CommentScore,CTMLabel,CotLabel,SO_Revisions_Link,verdict,Comment Type,Confidence,gt_label,pred_label
0,0,0,valid,invalid,73672870,You created batches of one element each by set...,You created batches of one element each by set...,2022-09-10 00:00:00 UTC,2022-09-12 11:17:50.63 UTC,14774959,...,"Just so I understand, can you show me how the ...",1,1,0,https://stackoverflow.com/questions/73672870/,0,Question,0.75,NO,NO
1,1,1,valid,valid,73603795,You are doing two different operations in your...,You are doing two different operations in your...,2022-09-05 00:00:00 UTC,2022-09-05 01:55:28.487 UTC,5854731,...,"Ok, Can you try this `test[[""ex1"",""ex2""]] = te...",1,1,0,https://stackoverflow.com/questions/73603795/,1,Solution,0.75,YES,YES
2,2,2,valid,valid,73694160,You need to set ServerName to the hostname in ...,You need to set ServerName to the hostname in ...,2022-09-12 00:00:00 UTC,2022-09-12 19:12:14.757 UTC,374642,...,"It worked! I changed the second line to ""Serve...",1,1,1,https://stackoverflow.com/questions/73694160/,1,Flaw,0.5,YES,NO
3,3,3,valid,invalid,11597911,The main advantages are that it makes multi-li...,The main advantages are that it makes multi-li...,2012-07-22 00:00:00 UTC,2022-09-02 21:12:10.99 UTC,4518341,...,This answer is really beautiful.,1,1,0,https://stackoverflow.com/questions/11597911/,0,Flaw,0.5,NO,NO
4,4,4,valid,invalid,73736507,There is probably a more efficient way to code...,There is probably a more efficient way to code...,2022-09-16 00:00:00 UTC,2022-09-16 15:33:31.75 UTC,16201558,...,@ouroboros1 true; I added an edit based on you...,1,1,1,https://stackoverflow.com/questions/73736507/,0,Solution,0.55,NO,YES


In [5]:
valid_df= df[df['pred_label']=='YES']

In [6]:
valid_df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'rag_pred', 'gt', 'PostId', 'code_before',
       'code_after', 'AcceptanceDate', 'LastEditDate', 'EditorId',
       'CommentDate', 'CommenterId', 'comment', 'CommentScore', 'CTMLabel',
       'CotLabel', 'SO_Revisions_Link', 'verdict', 'Comment Type',
       'Confidence', 'gt_label', 'pred_label'],
      dtype='object')

In [ ]:
#java(soup)
def generate_code(code_before, comment):
    prompt = f"""
    Consider the following java code from stack overflow answer:
    code_before: {code_before}

    Now consider the following comment and reflect the change in the code:
    Comment: {comment}
    
    Don't make any other changes. provide the fixed code only as adressed in the comment.
    """
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

# Generate fixed code using list comprehension
valid_df['updated_code'] = [
    generate_code(row['code_before'], row['comment'])
    for _, row in valid_df.iterrows()
]





In [ ]:
# python(pyvcp)
def generate_code(code_before, comment):
    prompt = f"""
    Consider the following python code from stack overflow answer:
    code_before: {code_before}

    Now consider the following comment and reflect the change in the code:
    Comment: {comment}
    
    Don't make any other changes. provide the fixed code only as adressed in the comment.
    """
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

# Generate fixed code using list comprehension
valid_df['updated_code'] = [
    generate_code(row['code_before'], row['comment'])
    for _, row in valid_df.iterrows()
]


/tmp/ipykernel_661945/4260178982.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['updated_code'] = [


In [ ]:
#valid_df.to_csv("/gpuhome/shanto1/Codellama_test/output/rag_reflection_codegen_java(soup).csv", index=False)

In [ ]:
valid_df.to_csv("", index=False) #output file

In [11]:
# select the two columns correctly (use a list of column names)
valid_df[['code_after', 'updated_code']]

,code_after,updated_code
1,You are doing two different operations in your...,"```python\ntest[[""ex1"", ""ex2""]] = test.apply(l..."
4,There is probably a more efficient way to code...,```python\ndf = pd.DataFrame({'vehicleID': {0:...
5,I might be wrong but this seems clear. In your...,"```python\ndef find_all_paths(graph, start, en..."
7,You can do that by changing the transparency a...,```python\nimport numpy as np\nimport matplotl...
8,Using .reset_index() to 'fill' your 'year' col...,```python\nimport pandas as pd\nimport matplot...
...,...,...
208,Issues in original question:\n\nYou can replac...,"```python\ndef extract_values(dct, lst=[], key..."
212,Here's a couple of suggestions:\n\nUse date_ra...,```python\nimport datetime\nimport pandas as p...
213,Your variables with _remainder are redundant.\...,```python\nchange_quarter = money // quarter\n...
214,Here's code making use of Pillow and Scipy's c...,```python\nimport struct\nfrom PIL import Imag...


In [27]:
valid_df.columns

Index(['PostHistoryId', 'PostId', 'code_before', 'code_after', 'CommentId',
       'comment', 'CommentCreationDate', 'response_time_min', 'gt_label',
       'rag_predict', 'pred_label', 'updated_code',
       'code_semantic_similarity'],
      dtype='object')

In [16]:
# Compute Exact Match, BLEU, and n-gram scores
valid_df['exact_match'] = valid_df.apply(
    lambda row: row['updated_code'] == row['code_after'], axis=1
)

In [ ]:
import math
from collections import Counter
from typing import List, Tuple
import pandas as pd


def tokenize(s: str) -> List[str]:
    # Simple whitespace tokenization (works well for code without extra deps)
    return s.strip().split()


def ngrams(tokens: List[str], n: int) -> List[Tuple[str, ...]]:
    return [tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]


def bleu_addone(candidate: str, reference: str, max_n: int = 4):
    """
    BLEU-N with add-one smoothing (Lin & Och 2004 style).
    Returns: bleu, [p1..pN], brevity_penalty, (len_cand, len_ref)
    """
    # Handle Nones/NaNs
    candidate = "" if candidate is None else str(candidate)
    reference = "" if reference is None else str(reference)

    cand_tokens = tokenize(candidate)
    ref_tokens = tokenize(reference)
    c, r = len(cand_tokens), len(ref_tokens)

    if c == 0:
        # No candidate tokens => 0 BLEU
        return 0.0, [0.0] * max_n, 0.0, (c, r)

    # Brevity penalty
    bp = 1.0 if c > r else math.exp(1 - float(r) / float(c))

    precisions = []
    for n in range(1, max_n + 1):
        cand_ngrams = ngrams(cand_tokens, n)
        ref_ngrams = ngrams(ref_tokens, n)
        cand_counts = Counter(cand_ngrams)
        ref_counts = Counter(ref_ngrams)
        match = sum(min(cnt, ref_counts.get(ng, 0)) for ng, cnt in cand_counts.items())
        total = sum(cand_counts.values())
        # add-one smoothing
        p_n = (match + 1.0) / (total + 1.0) if total > 0 else 0.0
        precisions.append(p_n)

    # Calculate log precisions, handling zero or negative values
    log_precisions = []
    # Use a very small number for log(0) - effectively negative infinity
    log_epsilon = math.log(1e-9)
    for p in precisions:
        if p > 0:
            log_precisions.append(math.log(p))
        else:
            log_precisions.append(log_epsilon) # Use log of a very small number for p <= 0

    # Ensure there are max_n log precisions, even if some n-grams were empty
    while len(log_precisions) < max_n:
         log_precisions.append(log_epsilon)


    geo_mean = math.exp(sum((1.0 / max_n) * log_p for log_p in log_precisions))
    bleu = bp * geo_mean
    return bleu, precisions, bp, (c, r)


def exact_match(a: str, b: str) -> float:
    return 1.0 if ("" if a is None else str(a)) == ("" if b is None else str(b)) else 0.0

# Use the existing valid_df DataFrame
df = pd.read_csv("") # Input file with 'code_after' and 'updated_code' columns
max_n = 1 # Use a default value for max_n


bleu_vals, p1s, ems = [], [], []

for _, row in df.iterrows():
    ref = row["code_after"]
    cand = row["updated_code"]
    bleu, ps, bp, _lens = bleu_addone(cand, ref, max_n=max_n)
    em = exact_match(cand, ref)

    bleu_vals.append(bleu)
    # pad precisions to 4 to avoid index errors if max_n != 4
    p1s.append(ps[0] if len(ps) > 0 else 0.0)
    ems.append(em)

df["bleu4_addone"] = bleu_vals
df["bleu_p1"] = p1s

df["em_score"] = ems # Renamed to avoid conflict with boolean column

# Print macro averages
avg_bleu = df["bleu4_addone"].mean()
avg_em = df["em_score"].mean()
print(f"Average BLEU-4 (add-one): {avg_bleu:.6f}")
print(f"Average Exact Match:      {avg_em:.6f}")

# Optionally, save the updated DataFrame
df.to_csv("valid_df_with_bleu.csv", index=False)
print("Wrote: valid_df_with_bleu.csv")

Average BLEU-4 (add-one): 0.716623
Average Exact Match:      0.093458
Wrote: valid_df_with_bleu.csv
